In [ ]:
arrow::read_parquet(here("data", "atpRaw.parquet"), as_data_frame = TRUE) -> atp
head(atp, 1)

,match.sets,match.tournamentRound,match.WL,opponent.name,opponent.rank,player.bornAt,player.hand,player.height,player.link,player.name,tournament.date,tournament.ground,tournament.location,tournament.name,tournament.prize
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,64 62,Semi-Finals,W,Marin Cilic,30,"Belgrade, Serbia","Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.22 - 2021.12.05,Hard,"Madrid, Spain",Davis Cup Finals,


Nota: estas transformações podiam ser melhoradas na sua eficiência, mas funcionam.

In [ ]:
# sets (promenores no transformacao)
atp %>% mutate(
    match.sets          = ifelse(str_detect(match.sets, "null") | str_trim(match.sets) == "", NA, match.sets),
    match.wasRet        = match.sets %>% str_detect("RET"),
    match.hadTieBreaker = match.sets %>% str_detect(","),
    match.wasWalkover   = match.sets %>% str_detect("W/O"),
    match.wasDef        = match.sets %>% str_detect("DEF")
) %>% 
    # https://regex101.com/r/XX1VNC/1
    mutate(match.sets = str_remove(match.sets, "(?i)\\(RET\\)|\\(W/O\\)|\\(DEF\\)|,") %>% str_squish()) %>%
    mutate(match.setsCount = str_count(match.sets, " ") + 1) %>%  
    mutate(
        match.setsCount = ifelse(
            !match.wasRet & !match.wasWalkover & !match.wasDef & match.setsCount == 1, #condition
            str_count(match.sets, "-")+1, #if yes 
            match.setsCount #if not
        ),
        match.wasRet = ifelse(
            !match.wasRet & !match.wasWalkover & !match.wasDef & match.setsCount == 1, #with new setsCount
            TRUE, #if it's still one after
            match.wasRet #dont change
        )
    ) -> atpS

In [ ]:
# countries (promoenores no transformacao)
read.csv("https://raw.githubusercontent.com/notPlancha/projeto-adb/main/data/countryAlias.csv") %>% 
    distinct() %>% 
    group_by(alias) %>% 
    summarise(country = first(country) %>% str_squish()) -> aliases
atpS %>%
    mutate(
        tournament.location = case_when(
            is.na(tournament.location) | str_trim(tournament.location) %in% c("TBA", "TBC", "TBD", "") ~ NA,
            str_detect(tournament.location, ",") ~ str_extract(tournament.location, "(?<=,\\s)[^,]+$") %>% str_squish(),
            TRUE ~ tournament.location %>% str_squish()
        ), player.bornAt = case_when(
            is.na(player.bornAt) | str_trim(player.bornAt) %in% c("TBA", "TBC", "TBD", "") ~ NA,
            str_detect(player.bornAt, ",") ~ str_extract(player.bornAt, "(?<=,\\s)[^,]+$") %>% str_squish(),
            TRUE ~ player.bornAt %>% str_squish()
        )
    ) %>% 
    left_join(aliases, by = join_by(tournament.location == alias)) %>% 
    mutate(tournament.location = country) %>% select(-country) %>%
    left_join(aliases, by = join_by(player.bornAt == alias)) %>%
    mutate(player.bornAt = country) %>% select(-country) -> atpSC

In [ ]:
atpSC

match.sets,match.tournamentRound,match.WL,opponent.name,opponent.rank,player.bornAt,player.hand,player.height,player.link,player.name,tournament.date,tournament.ground,tournament.location,tournament.name,tournament.prize,match.wasRet,match.hadTieBreaker,match.wasWalkover,match.wasDef,match.setsCount
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>
64 62,Semi-Finals,W,Marin Cilic,30,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.22 - 2021.12.05,Hard,Spain,Davis Cup Finals,,FALSE,FALSE,FALSE,FALSE,2
63 76,Round of 16,W,Karen Khachanov,26,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2022.02.21 - 2022.02.26,Hard,United Arab Emirates,Dubai,"$2,794,840",FALSE,FALSE,FALSE,FALSE,2
62 61,Round Robin,W,Cameron Norrie,12,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.15 - 2021.11.21,Hard,Italy,Nitto ATP Finals,"$7,250,000",FALSE,FALSE,FALSE,FALSE,2
63 62,Round Robin,W,Andrey Rublev,5,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.15 - 2021.11.21,Hard,Italy,Nitto ATP Finals,"$7,250,000",FALSE,FALSE,FALSE,FALSE,2
46 63 63,Finals,W,Daniil Medvedev,2,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.01 - 2021.11.07,Hard,France,ATP Masters 1000 Paris,"�5,207,405",FALSE,FALSE,FALSE,FALSE,3
36 60 76,Semi-Finals,W,Hubert Hurkacz,10,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.01 - 2021.11.07,Hard,France,ATP Masters 1000 Paris,"�5,207,405",FALSE,FALSE,FALSE,FALSE,3
76 62,Round Robin,W,Casper Ruud,8,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.15 - 2021.11.21,Hard,Italy,Nitto ATP Finals,"$7,250,000",FALSE,TRUE,FALSE,FALSE,2
63 62,Round Robin,W,Dennis Novak,118,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2021.11.22 - 2021.12.05,Hard,Spain,Davis Cup Finals,,FALSE,FALSE,FALSE,FALSE,2
46 67,Quarter-Finals,L,Jiri Vesely,123,Serbia,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,Novak Djokovic,2022.02.21 - 2022.02.26,Hard,United Arab Emirates,Dubai,"$2,794,840",FALSE,FALSE,FALSE,FALSE,2


In [ ]:
# players
(atpSC %>%
    filter(str_trim(opponent.name) != "bye") %>%
    mutate(
        player.id = str_extract(player.link, r"{(?<=/)\w\w\w\w(?=/player-activity)}"),
        player.hand = ifelse(str_trim(player.hand) == "null", NA, str_trim(player.hand))
    ) %>% 
    group_by(player.id, player.name) %>% 
    summarise(
        player.bornAt = first(player.bornAt),
        player.hand   = first(player.hand  ),
        player.height = first(player.height),
    ) %>%
    separate_wider_delim(player.hand, delim=",", names=c("player.domHand","player.backHand")) -> players) %>%
    mutate(player.domHand = str_squish(player.domHand), player.backHand = str_squish(player.backHand))

`summarise()` has grouped output by 'player.id'. You can override using the `.groups` argument.


player.id,player.name,player.bornAt,player.domHand,player.backHand,player.height
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
a005,Nelson Aerts,Brazil,Right-Handed,Unknown Backhand,188
a006,Ronald Agenor,Morocco,Right-Handed,Unknown Backhand,180
a007,Juan Aguilera,Spain,Right-Handed,Unknown Backhand,183
a018,Christer Allgardh,Sweden,Right-Handed,Unknown Backhand,191
a022,Vijay Amritraj,India,Right-Handed,Unknown Backhand,193
a025,Matt Anger,USA,Right-Handed,Unknown Backhand,188
a027,Paul Annacone,USA,Right-Handed,Unknown Backhand,185
a028,Alex Antonitsch,Austria,Right-Handed,Unknown Backhand,188
a030,Roberto Arguello,Argentina,NA,NA,173


In [ ]:
# matches
atpSC %>%
    filter(str_trim(opponent.name) != "bye") %>%
    mutate(
        player.id = str_extract(player.link, r"{(?<=/)\w\w\w\w(?=/player-activity)}"),
        player.hand = ifelse(str_trim(player.hand) == "null", NA, str_trim(player.hand))
    ) %>%
    mutate(
        winner.name     = ifelse(match.WL == "W", player.name    , opponent.name  ),
        looser.name     = ifelse(match.WL == "W", opponent.name  , player.name    ),
        winner.id       = ifelse(match.WL == "W", player.id      , NA             ),
        looser.id       = ifelse(match.WL == "W", NA             , player.id      ),
        winner.rank     = ifelse(match.WL == "W", NA             , opponent.rank  ), # ta trocado na db
        looser.rank     = ifelse(match.WL == "W", opponent.rank  , NA             ),
    ) %>%
    group_by(
        tournament.name, 
        tournament.date, 
        match.tournamentRound, 
        winner.name, winner.id, 
        looser.name, looser.id
    ) %>%
    summarise(
        temp.count = n(),
        match.setsCount     = first(match.setsCount    ),
        match.wasRet        = first(match.wasRet       ),
        match.hadTieBreaker = first(match.hadTieBreaker),
        match.wasWalkover   = first(match.wasWalkover  ),
        match.wasDef        = first(match.wasDef       ),
        tournament.prize    = first(tournament.prize   ),
        tournament.ground   = first(tournament.ground  ),
        tournament.location = first(tournament.location),
        winner.rank         = first(winner.rank),
        looser.rank         = first(looser.rank),
    ) -> matches
matches

`summarise()` has grouped output by 'tournament.name', 'tournament.date', 'match.tournamentRound', 'winner.name',
'winner.id', 'looser.name'. You can override using the `.groups` argument.


tournament.name,tournament.date,match.tournamentRound,winner.name,winner.id,looser.name,looser.id,temp.count,match.setsCount,match.wasRet,match.hadTieBreaker,match.wasWalkover,match.wasDef,tournament.prize,tournament.ground,tournament.location,winner.rank,looser.rank
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
's-Hertogenbosch,1998.06.15 - 1998.06.21,Finals,Patrick Rafter,r255,Martin Damm,NA,1,2,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands,NA,85
's-Hertogenbosch,1998.06.15 - 1998.06.21,Finals,Patrick Rafter,NA,Martin Damm,d214,1,2,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands,6,NA
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Dennis Van Scheppingen,s559,Richard Krajicek,NA,1,1,FALSE,FALSE,TRUE,FALSE,"$475,000",Grass,Netherlands,NA,12
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Dennis Van Scheppingen,NA,Richard Krajicek,k214,1,1,FALSE,FALSE,TRUE,FALSE,"$475,000",Grass,Netherlands,126,NA
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Jan Siemerink,s381,Steve Campbell,NA,1,3,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands,NA,95
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Jan Siemerink,NA,Steve Campbell,c355,1,3,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands,26,NA
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Martin Damm,d214,Karol Kucera,NA,1,3,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands,NA,11
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Martin Damm,NA,Karol Kucera,k237,1,3,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands,85,NA
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Patrick Rafter,r255,Sjeng Schalken,NA,1,2,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands,NA,55


Esta tabela inclui todos os jogos das duas prespectivas, ou de apenas uma das prespectivas se a outra não estiver presentes. Este primeiro caso apenas é necessário juntar, enquanto que o segundo caso precisa de uma decisão. Sendo este caso significa que os jogadores não têm informação necessária para a nossa análise (como a idade ou onde nasceram), podemos tentar não usar estes jogos. Se concluirmos que não temos dados suficientes para o treino mais à frente, podemos adaptar a nossa solução para os incluir. A esclusão destes dados significa que o nosso modelo, sem informação dos jogadores, não vai conseguir corretamente prever o número de sets do jogo.

In [ ]:
matches %>% 
    group_by(
        tournament.name, 
        tournament.date,
        match.tournamentRound, 
        winner.name, 
        looser.name,
    ) %>% summarise(
        winner.id = first(winner.id, na_rm=T),
        looser.id = first(looser.id, na_rm=T),
        match.setsCount     = first(match.setsCount    ),
        match.wasRet        = first(match.wasRet       ),
        match.hadTieBreaker = first(match.hadTieBreaker),
        match.wasWalkover   = first(match.wasWalkover  ),
        match.wasDef        = first(match.wasDef       ),
        tournament.prize    = first(tournament.prize   ),
        tournament.ground   = first(tournament.ground  ),
        tournament.location = first(tournament.location),
    ) -> matchesF
matchesF

`summarise()` has grouped output by 'tournament.name', 'tournament.date', 'match.tournamentRound', 'winner.name'. You can
override using the `.groups` argument.


tournament.name,tournament.date,match.tournamentRound,winner.name,looser.name,winner.id,looser.id,match.setsCount,match.wasRet,match.hadTieBreaker,match.wasWalkover,match.wasDef,tournament.prize,tournament.ground,tournament.location
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>
's-Hertogenbosch,1998.06.15 - 1998.06.21,Finals,Patrick Rafter,Martin Damm,r255,d214,2,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Dennis Van Scheppingen,Richard Krajicek,s559,k214,1,FALSE,FALSE,TRUE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Jan Siemerink,Steve Campbell,s381,c355,3,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Martin Damm,Karol Kucera,d214,k237,3,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Quarter-Finals,Patrick Rafter,Sjeng Schalken,r255,s572,2,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Round of 16,Dennis Van Scheppingen,Peter Wessels,s559,w254,2,FALSE,TRUE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Round of 16,Jan Siemerink,Filip Dewulf,s381,d228,2,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Round of 16,Karol Kucera,Kenneth Carlsen,k237,c328,2,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands
's-Hertogenbosch,1998.06.15 - 1998.06.21,Round of 16,Martin Damm,John van Lottum,d214,v201,2,FALSE,FALSE,FALSE,FALSE,"$475,000",Grass,Netherlands


In [ ]:
# join players
matchesFP <- matchesF %>%
    left_join(players, by = join_by(winner.id == player.id)) %>%
    rename(
        winner.bornAt = player.bornAt, 
        winner.domHand = player.domHand, 
        winner.backHand = player.backHand, 
        winner.height = player.height
    ) %>% 
    select(-player.name) %>%
    left_join(players, by = join_by(looser.id == player.id)) %>%
    rename(
        looser.bornAt = player.bornAt, 
        looser.domHand = player.domHand, 
        looser.backHand = player.backHand, 
        looser.height = player.height
    ) %>% 
    select(-player.name) %>% relocate(starts_with("winner"), starts_with("looser"), starts_with("match"), starts_with("tournament"))
matchesFP %>% glimpse()

Rows: 682,069
Columns: 23
Groups: tournament.name, tournament.date, match.tournamentRound, winner.name [678,502]
$ winner.name           <chr> "Patrick Rafter", "Dennis Van Scheppingen", "Jan…
$ winner.id             <chr> "r255", "s559", "s381", "d214", "r255", "s559", …
$ winner.bornAt         <chr> "Australia", "Netherlands", "Netherlands", "Czec…
$ winner.domHand        <chr> "Right-Handed", "Right-Handed", "Left-Handed", "…
$ winner.backHand       <chr> " One-Handed Backhand", " Unknown Backhand", " U…
$ winner.height         <chr> "185", "185", "183", "188", "185", "185", "183",…
$ looser.name           <chr> "Martin Damm", "Richard Krajicek", "Steve Campbe…
$ looser.id             <chr> "d214", "k214", "c355", "k237", "s572", "w254", …
$ looser.bornAt         <chr> "Czech Republic", "Netherlands", "USA", "Czech R…
$ looser.domHand        <chr> "Right-Handed", "Right-Handed", "Right-Handed", …
$ looser.backHand       <chr> " Unknown Backhand", " One-Handed Backhand", " U…
$ loose

In [ ]:
# dates (promoenores no transformacao)
matchesFPD <- matchesFP %>% ungroup() %>%
    mutate(tournament.date = case_when(
        tournament.date == "1972.10.16 - 1972.10.15" ~ "1972.10.09 - 1972.10.15",
        tournament.date == "1983.05.02 - 1973.05.02" ~ "1983.05.02 - 1983.05.08",
        tournament.date == "1992.08.10 - 1992.08.09" ~ "1992.08.09 - 1992.08.09",
        TRUE ~ tournament.date
    )) %>%
    separate_wider_delim(tournament.date, " - ", names=c("ds", "df"), too_few = "align_start") %>% 
    mutate(tournament.date_start = ymd(ds), tournament.date_finish = ymd(df)) %>%
    mutate(tournament.duration = ifelse(
        tournament.date_finish %>% is.na(),
        0,
        tournament.date_finish - tournament.date_start
    ))
matchesFPD

winner.name,winner.id,winner.bornAt,winner.domHand,winner.backHand,winner.height,looser.name,looser.id,looser.bornAt,looser.domHand,⋯,match.wasDef,tournament.name,ds,df,tournament.prize,tournament.ground,tournament.location,tournament.date_start,tournament.date_finish,tournament.duration
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>
Patrick Rafter,r255,Australia,Right-Handed,One-Handed Backhand,185,Martin Damm,d214,Czech Republic,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Dennis Van Scheppingen,s559,Netherlands,Right-Handed,Unknown Backhand,185,Richard Krajicek,k214,Netherlands,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Jan Siemerink,s381,Netherlands,Left-Handed,Unknown Backhand,183,Steve Campbell,c355,USA,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Martin Damm,d214,Czech Republic,Right-Handed,Unknown Backhand,188,Karol Kucera,k237,Czech Republic,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Patrick Rafter,r255,Australia,Right-Handed,One-Handed Backhand,185,Sjeng Schalken,s572,Netherlands,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Dennis Van Scheppingen,s559,Netherlands,Right-Handed,Unknown Backhand,185,Peter Wessels,w254,Netherlands,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Jan Siemerink,s381,Netherlands,Left-Handed,Unknown Backhand,183,Filip Dewulf,d228,Belgium,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Karol Kucera,k237,Czech Republic,Right-Handed,Two-Handed Backhand,185,Kenneth Carlsen,c328,Denmark,Left-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6
Martin Damm,d214,Czech Republic,Right-Handed,Unknown Backhand,188,John van Lottum,v201,Madagascar,Right-Handed,⋯,FALSE,'s-Hertogenbosch,1998.06.15,1998.06.21,"$475,000",Grass,Netherlands,1998-06-15,1998-06-21,6


In [ ]:
matchesFPD %>% write_csv(here("data", "atpTrans.csv")) # tava a crashar se fosse direto https://github.com/apache/arrow/issues/34923 (2/5/2023, arrow v11.0.0.3)
arrow::read_csv_arrow(here("data", "atpTrans.csv")) %>% arrow::write_parquet(here("data", "atpTrans.parquet"))